## Necessary packages & configurations

In [15]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import source_code as sc #source code file with functions
import importlib #for resetting library

import tensorflow as tf
import keras

#Reproducibility
np.random.seed(12345)

hdf5file = 'data/dataset.hdf5'

print(tf.__version__)
print(keras.__version__)

1.14.0
2.2.5


In [16]:
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import (Dense,
                          Activation,
                          Conv3D,
                          MaxPooling3D,
                          ZeroPadding3D,
                          Flatten,
                          Dropout)

In [17]:
import warnings
warnings.filterwarnings("ignore")
import os
import datetime
now = datetime.datetime.now()
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"

#LOIC: IS THIS RANDOM SELECTING SOMETHING?
igpu = now.second%2; print(igpu)
if igpu==0: 
     os.environ["CUDA_VISIBLE_DEVICES"]="0" 
else:
     os.environ["CUDA_VISIBLE_DEVICES"]="1"

1


## Reading in Data and Randomise

In [18]:
importlib.reload(sc)
#note: test data will only be loaded when model has to be tested
x_train,y_train = sc.ReadFile(hdf5file ,split=False)
x_test,y_test = sc.ReadFile(hdf5file,data='test',split=False)
#Rotate x_train randomly
x_train,rotatevector = sc.RandomRotateSamples(x_train)

#Scale the Dataset
x_train,x_test = sc.Scale(x_train,x_test)

In [19]:
#Approx. mean 0 and stdev of 1. Should be sufficient

print('> SHAPE: ',x_train.shape)
print('> MEAN: ',np.mean(x_train,axis=(0,1,2,3)))
print('> STDEV: ',np.std(x_train,axis=(0,1,2,3)))

> SHAPE:  (25600, 32, 32, 32, 3)
> MEAN:  [0.16103153 0.16       0.16006407]
> STDEV:  [0.98314345 1.1313708  1.1313862 ]


## Model-Building

In [20]:
#Starting of with a sequential model
input_shape = x_train.shape[1:]
nclass = np.unique(y_train).size

In [22]:
#Model of Stefano but on randomised data
model = Sequential()
model.add(Conv3D(16, (3, 3, 3), activation='relu', 
                        padding='same', name='conv1',
                        strides=(1, 1, 1), 
                        input_shape=input_shape))
model.add(MaxPooling3D(pool_size=(2, 2, 2), 
                       strides=(2, 2, 2), 
                       padding='same', 
                       name='pool1'))
model.add(Conv3D(32, (3, 3, 3), activation='relu', 
                        padding='same', name='conv2',
                        strides=(1, 1, 1)))
model.add(MaxPooling3D(pool_size=(2, 2, 2), 
                       strides=(2, 2, 2), 
                       padding='same', 
                       name='pool2'))
model.add(Conv3D(64, (3, 3, 3), activation='relu', 
                        padding='same', name='conv3a',
                        strides=(1, 1, 1)))
model.add(Conv3D(64, (3, 3, 3), activation='relu', 
                        padding='same', name='conv3b',
                        strides=(1, 1, 1)))
model.add(MaxPooling3D(pool_size=(2, 2, 2), 
                       strides=(2, 2, 2), 
                       padding='same', 
                       name='pool3'))
model.add(Conv3D(128, (3, 3, 3), activation='relu', 
                        padding='same', name='conv4a',
                        strides=(1, 1, 1)))
model.add(Conv3D(128, (3, 3, 3), activation='relu', 
                        padding='same', name='conv4b',
                        strides=(1, 1, 1)))
model.add(MaxPooling3D(pool_size=(2, 2, 2), 
                       strides=(2, 2, 2), 
                       padding='same', 
                       name='pool4'))
model.add(Conv3D(128, (3, 3, 3), activation='relu', 
                        padding='same', name='conv5a',
                        strides=(1, 1, 1)))
model.add(Conv3D(128, (3, 3, 3), activation='relu', 
                        padding='same', name='conv5b',
                        strides=(1, 1, 1)))
model.add(ZeroPadding3D(padding=(1, 1, 1)))
model.add(MaxPooling3D(pool_size=(2, 2, 2), 
                       strides=(2, 2, 2), 
                       padding='same', 
                       name='pool5'))
model.add(Flatten())
model.add(Dense(1028, activation='relu', name='fc6'))
model.add(Dropout(.5))
model.add(Dense(1028, activation='relu', name='fc7'))
model.add(Dropout(.5))
model.add(Dense(nclass, activation='softmax', name='fc8'))





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
optimizer = 